# Import Data to the online web app

This is the notebook to update the arcgis instant app
This requires arcpy as a package
Before running this notebook, you should have already set up the acrGIS instant app. This notebook only replaces the shapefile already in the map, it does nto create a new map

In [1]:
from arcgis.gis import GIS
import os

To create the GIS object, we pass your profile that contains the url and your login credentials. Please replace the credentials below with that of your org. 
To learn more about profiles, see [here](https://developers.arcgis.com/python/guide/working-with-different-authentication-schemes/#Storing-your-credentialls-locally).

In [2]:
gis = GIS('home')


#If this does not automatically connect, you may need to do this code instead:
#gis = GIS(
#  url="https://www.arcgis.com",
#  username="username",
#  password=getpass.getpass("Enter password:")
#)

# or do this:
# gis = GIS("https://www.arcgis.com", "your_username", "your_password")

## Delete the former shapefile so we can upload the new one

In [9]:
# search for the layer id
search_result = gis.content.search('title:FIM', item_type = 'Feature Service')
display(search_result)

[<Item title:"FIM" type:Feature Layer Collection owner:cwatts_UMass_Amherst>]

In [10]:
# Replace 'item_id' with the actual ID of the item you want to delete
item_id = search_result[0].id
item_to_delete = gis.content.get(item_id)

# Delete the item, remove it permanently is necesary if you have a recycle bin enabled at your organization
item_to_delete.delete(permanent=True)

True

In [11]:
# search for the shapefile ID
search_result = gis.content.search('title:FIM', item_type = 'Shapefile')
display(search_result)

# Replace 'item_id' with the actual ID of the item you want to delete
item_id = search_result[0].id
item_to_delete = gis.content.get(item_id)

# Delete the item, remove it permanently is necesary if you have a recycle bin enabled at your organization
item_to_delete.delete(permanent=True)


[<Item title:"FIM" type:Shapefile owner:cwatts_UMass_Amherst>]

True

## upload the new shapefile to ArcGIS Online
edit the metadata with the title and tags of the file. the path must lead to a zip file

In [13]:
# get datetime to tag the fiel name
from datetime import datetime

current_datetime = datetime.now()
print(current_datetime)

# Format the datetime as a string (e.g., "2023-04-01 12:00")
formatted_datetime = current_datetime.strftime('%Y-%m-%d %H:%M')

2024-07-12 14:09:02.594439


In [14]:
# edit the dictionary with the title and tags you want
metadata = {
    'title': f'FIM {formatted_datetime}',
    'tags': 'Flood Forecast, 5 Days, Inundation, National Water Model',
    'type': 'Shapefile'
}
data_path = os.path.join('C:/Users/Lyn/Desktop/Berylb.zip')


In [15]:
shapefile = gis.content.add(metadata, data=data_path)

In [16]:
feature_layer = shapefile.publish()

In [17]:
feature_layer.url

'https://services.arcgis.com/2gdL2gxYNFY2TOUb/arcgis/rest/services/Berylb/FeatureServer'

##  Add the layer to the webmap Instant App

In [22]:
from arcgis.mapping import WebMap, WebScene
from IPython.display import display
import json


# ID the web map you want to alter
search_result = gis.content.search("title:Flood Forecast Houston", item_type = "Web Map")

# read the webmap as an object
wm_item = search_result[0]
web_map_obj = WebMap(wm_item)



In [23]:
# check the layers on the web map
for lyr in web_map_obj.layers:
    print(lyr.title + " " + lyr.url)

Beryl https://services.arcgis.com/2gdL2gxYNFY2TOUb/arcgis/rest/services/Beryla/FeatureServer/0
Beryl https://services.arcgis.com/2gdL2gxYNFY2TOUb/arcgis/rest/services/Beryla/FeatureServer/0
Beryl https://services.arcgis.com/2gdL2gxYNFY2TOUb/arcgis/rest/services/Beryla/FeatureServer/0


In [24]:
# Feature service item id for the missing layer:
web_map_obj.layers[0]["itemId"]

'2c0af85fc05541d9a429be84e9c9e7be'

to do: have the title change automatically to fit the title above

In [25]:

# search for the newly uploaded layer
search_result = gis.content.search('title:Berylb', item_type = 'Feature Service')
display(search_result)

# check the item id
search_result[0].id

[<Item title:"FIM 2024-07-12 14:09" type:Feature Layer Collection owner:cwatts_UMass_Amherst>]

'd0f6e18ccee14e6abf88898bb2068cb2'

In [26]:
# update the web map with the new layer
capitals = search_result[0]
capitals.layers

[<FeatureLayer url:"https://services.arcgis.com/2gdL2gxYNFY2TOUb/arcgis/rest/services/Berylb/FeatureServer/0">]

to do: edit the title to match above

In [27]:
# The new feature service does have a layer with id 0. Hence we can use the same layer id while switching the url. 
# To remove the old layer, call remove_layer() method. 
# Then add the correct FeatureLayer object by calling the add_layer() method on the WebMap object.

# remove the old layer from the web map
web_map_obj.remove_layer(web_map_obj.layers[0])

# add the correct layer. While adding you can customize the title
web_map_obj.add_layer(capitals.layers[0], options={'title':'Berylb'})

#check the layers on the web map
for lyr in web_map_obj.layers:
    print(lyr.title + " " + lyr.url)

# update the web map 
web_map_obj.update(item_properties={'title':'USA Capitals - updated'})



Beryl https://services.arcgis.com/2gdL2gxYNFY2TOUb/arcgis/rest/services/Beryla/FeatureServer/0
Beryl https://services.arcgis.com/2gdL2gxYNFY2TOUb/arcgis/rest/services/Beryla/FeatureServer/0
Berylb https://services.arcgis.com/2gdL2gxYNFY2TOUb/arcgis/rest/services/Berylb/FeatureServer/0


True

In [28]:
# Let's clean it up so we can always run this section again
wm_item.delete()
#new_capitals.delete()
#csv_item.delete()

True